In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
def save_file(df,filename,table_id):
    if not os.path.exists('../Output Files/Collated Data/Collated Table'+table_id):
        os.mkdir('../Output Files/Collated Data/Collated Table'+table_id)
    
    df.to_csv('../Output Files/Collated Data/Collated Table'+table_id+'/'+filename+".csv",index=False)

In [3]:
diseases ={
          'LPS_Cord':'LPS_Cord',
          }

In [4]:
for disease in diseases.items():
    print(disease[0])
    df = pd.read_csv("../Output Files/TR List/"+disease[1]+".csv")[['term','genes','logFC']]
    df.rename(columns={'logFC':'logFC_TR'},inplace=True)
    df_temp = df[['term','genes']]
    df_temp["index1"] = df_temp.index
    df_temp = (df_temp.set_index(['index1','term'])
    .stack()
    .str.split(';',expand=True)
    .stack()
    .unstack(-2)
    .reset_index(-1,drop = True)
    .reset_index()
    .drop("index1",axis=1))
    
    df = df.merge(df_temp, how="left",  left_on='term', right_on='term').drop("genes_x",axis=1).rename(columns={'genes_y':'genes'})
    
    df_deg = pd.read_csv("../Sample Files/DEG/"+disease[0]+'.csv')[["Refseq_ID","logFC"]].rename(columns={'Refseq_ID':'genes','logFC':'logFC_gene'})
    df = df.merge(df_deg, how="left",  left_on='genes', right_on='genes')
    df["genes"] = df["genes"].str.capitalize().str.strip()
    df.replace("",np.nan,inplace=True)
    df.dropna(axis=0,subset=["genes"],inplace=True)
    df.drop_duplicates(subset=["term","genes"],inplace=True)
    save_file(df[["term","genes","logFC_TR","logFC_gene"]] ,disease[1],'2')

LPS_Cord


In [5]:
def calc_activation_state(df, i, cutoff_percent, df_direction):
    positive = len(df_direction[df_direction>0])
    negative = len(df_direction[df_direction<0])
    total = len(df_direction)
    if (positive/total)>cutoff_percent:
        df.loc[i,'Direction-'+str(cutoff_percent*100)+'%'] = '+'
        df.loc[i,'Activation State-'+str(cutoff_percent*100)+'%'] = len(df_direction[df_direction>0])/len(target_genes)
    elif (negative/total)>cutoff_percent:
        df.loc[i,'Direction-'+str(cutoff_percent*100)+'%'] = '-'
        df.loc[i,'Activation State-'+str(cutoff_percent*100)+'%'] = -1*len(df_direction[df_direction<0])/len(target_genes)
    else:
        df.loc[i,'Direction-'+str(cutoff_percent*100)+'%'] = '='
        df.loc[i,'Activation State-'+str(cutoff_percent*100)+'%'] = 0

In [6]:
for disease in diseases.items():
    print(disease[0])
    df_deg = pd.read_csv("../Sample Files/DEG/"+disease[0]+'.csv')[["Refseq_ID","logFC"]].rename(columns={'Refseq_ID':'genes','logFC':'logFC_gene'})
    len_deg = len(df_deg)
    df = pd.read_csv("../Output Files/TR List/"+disease[1]+".csv")
    df.rename(columns={'logFC':'logFC_TR'},inplace=True)
    df['weight'] = 0
    df_collated = pd.read_csv('../Output Files/Collated Data/Collated Table2/'+disease[0]+".csv")
    df_collated['direction'] = -1
    df_collated['direction'][df_collated['logFC_gene']>0] = 1

    for i in range(len(df)):
        target_genes = df.iloc[i]['genes'].split(';')
        df_direction = df_collated['direction'][df_collated['term']==df.iloc[i]['term']]
        direction = df_direction.sum()
        df.loc[i,'weight'] = len(target_genes)/len_deg
        calc_activation_state(df, i, 0.5, df_direction)
        calc_activation_state(df, i, 0.66, df_direction)
        calc_activation_state(df, i, 0.75, df_direction)
    save_file(df ,disease[1],'1')

LPS_Cord


c:\python36\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
